In [ ]:
!pip install -r requirements.txt

In [ ]:
from collections.abc import Callable
from enum import Enum

from numpy import full, ndarray, zeros
from scipy.linalg import solve_banded


class Method(Enum):
    ALTERNATING_DIRECTION = 0,
    FRACTIONAL_STEP = 1


def splitting_method(alpha: ndarray, f_x_y_t: Callable,
        l_upper: ndarray, n_upper: tuple, t_upper: float, k_upper: int,
        alpha1: float, beta1: float, alpha2: float, beta2: float,
        alpha3: float, beta3: float, alpha4: float, beta4: float,
        phi_1_x_t: Callable, phi_2_x_t: Callable,
        phi_3_y_t: Callable, phi_4_y_t: Callable,
        initial: Callable, method: Method, callback: Callable) -> ndarray:
    h, tau = l_upper / n_upper, t_upper / k_upper
    sigma = alpha * tau / (h * h)

    ai, ci = full((2, n_upper[0] + 1),
        sigma[0] / (2.0 if method == Method.ALTERNATING_DIRECTION else 1.0))
    bi = full(n_upper[0] + 1,
        -(1.0 if method == Method.ALTERNATING_DIRECTION else 2.0) * sigma[0] - 1.0)
    aj, cj = full((2, n_upper[1] + 1),
        sigma[1] / (2.0 if method == Method.ALTERNATING_DIRECTION else 1.0))
    bj = full(n_upper[1] + 1,
        -(1.0 if method == Method.ALTERNATING_DIRECTION else 2.0) * sigma[1] - 1.0)
    di, dj = zeros(n_upper[0] + 1), zeros(n_upper[1] + 1)

    ai[0] = ci[-1] = aj[0] = cj[-1] = 0.0
    bi[0] = beta3 - alpha3 / h[0]
    ci[0] = alpha3 / h[0]
    ai[-1] = -alpha4 / h[0]
    bi[-1] = beta4 + alpha4 / h[0]
    bj[0] = beta1 - alpha1 / h[1]
    cj[0] = alpha1 / h[1]
    aj[-1] = -alpha2 / h[1]
    bj[-1] = beta2 + alpha2 / h[1]

    u, u_temp = zeros((2, n_upper[0] + 1, n_upper[1] + 1))
    __initial_approximation(n_upper, h, initial, u)
    callback(u)
    for k in range(1, k_upper + 1):
        __step_x(f_x_y_t, n_upper, h, tau, k, sigma,
            alpha1, beta1, alpha2, beta2,
            phi_1_x_t, phi_2_x_t, phi_3_y_t, phi_4_y_t, ai, bi, ci, di,
            method, u, u_temp)
        __step_y(f_x_y_t, n_upper, h, tau, k, sigma,
            alpha3, beta3, alpha4, beta4,
            phi_1_x_t, phi_2_x_t, phi_3_y_t, phi_4_y_t, aj, bj, cj, dj,
            method, u_temp, u)
        callback(u)
    return u


def __initial_approximation(n_upper, h, initial, u):
    for i in range(n_upper[0] + 1):
        for j in range(n_upper[1] + 1):
            u[i, j] = initial(i * h[0], j * h[1])
    return u


def __step_x(f_x_y_t: Callable,
        n_upper: tuple, h: tuple, tau: float, k: int, sigma: ndarray,
        alpha1: float, beta1: float, alpha2: float, beta2: float,
        phi_1_x_t, phi_2_x_t, phi_3_y_t, phi_4_y_t,
        ai: ndarray, bi: ndarray, ci: ndarray, di: ndarray,
        method: Method, u_k_minus_1: ndarray, u_k_minus_1_divides_2: ndarray):
    t: float = (k - 0.5) * tau
    for j in range(1, n_upper[1]):
        di[0], di[-1] = phi_3_y_t(j * h[1], t), phi_4_y_t(j * h[1], t)
        for i in range(1, n_upper[0]):
            di[i] = (-u_k_minus_1[i, j]
                - tau / 2.0 * f_x_y_t(i * h[0], j * h[1], t))
            if method == Method.ALTERNATING_DIRECTION:
                di[i] += (sigma[1] * u_k_minus_1[i, j]
                    - sigma[1] / 2.0 * u_k_minus_1[i, j - 1]
                    - sigma[1] / 2.0 * u_k_minus_1[i, j + 1])
        u_k_minus_1_divides_2[:, j] = thomas(ai, bi, ci, di)
    for i in range(0, n_upper[0] + 1):
        u_k_minus_1_divides_2[i, 0] = (-alpha1 * u_k_minus_1_divides_2[i, 1]
                + h[1] * phi_1_x_t(i * h[0], t)) / (beta1 * h[1] - alpha1)
        u_k_minus_1_divides_2[i, -1] = (alpha2 * u_k_minus_1_divides_2[i, -2]
                + h[1] * phi_2_x_t(i * h[0], t)) / (beta2 * h[1] + alpha2)


def __step_y(f_x_y_t: Callable,
        n_upper: tuple, h: tuple, tau: float, k: int, sigma: ndarray,
        alpha3: float, beta3: float, alpha4: float, beta4: float,
        phi_1_x_t, phi_2_x_t, phi_3_y_t, phi_4_y_t,
        aj: ndarray, bj: ndarray, cj: ndarray, dj: ndarray,
        method: Method, u_k_minus_1_divides_2: ndarray, u_k: ndarray):
    t: float = k * tau
    for i in range(1, n_upper[0]):
        dj[0], dj[-1] = phi_1_x_t(i * h[0], t), phi_2_x_t(i * h[0], t)
        for j in range(1, n_upper[1]):
            dj[j] = (-u_k_minus_1_divides_2[i, j]
                - tau / 2.0 * f_x_y_t(i * h[0], j * h[1], t))
            if method == Method.ALTERNATING_DIRECTION:
                dj[j] += (sigma[0] * u_k_minus_1_divides_2[i, j]
                    - sigma[0] / 2.0 * u_k_minus_1_divides_2[i - 1, j]
                    - sigma[0] / 2.0 * u_k_minus_1_divides_2[i + 1, j])
        u_k[i, :] = thomas(aj, bj, cj, dj)
    for j in range(0, n_upper[1] + 1):
        u_k[0, j] = (-alpha3 * u_k[1, j]
            + h[0] * phi_3_y_t(j * h[1], t)) / (beta3 * h[0] - alpha3)
        u_k[-1, j] = (alpha4 * u_k[-2, j]
            + h[0] * phi_4_y_t(j * h[1], t)) / (beta4 * h[0] + alpha4)

In [ ]:
from math import cos, sin


def f_x_y_t(a: float, b: float, mu: float,
        x: float, y: float, t: float) -> float:
    return sin(x) * sin(y) * (MU * cos(MU * t) + (a + b) * sin(MU * t))


def phi_1_x_t(a: float, b: float, mu: float,
        x: float, t: float) -> float:
    return 0.0


def phi_2_x_t(a: float, b: float, mu: float,
        x: float, t: float) -> float:
    return -sin(x) * sin(mu * t)


def phi_3_y_t(a: float, b: float, mu: float,
        y: float, t: float) -> float:
    return 0.0


def phi_4_y_t(a: float, b: float, mu: float,
        y: float, t: float) -> float:
    return -sin(y) * sin(mu * t)

                
def psi_x_y(a: float, b: float, mu: float,
        x: float, y: float) -> float:
    return 0.0


def u_exact(a: float, b: float, mu: float,
        x: float, y: float, t: float) -> float:
    return sin(x) * sin(y) * sin(mu * t)

In [ ]:
from math import pi
from numpy import array

L = array([pi, pi])
ALPHA1, BETA1 = 0.0, 1.0
ALPHA2, BETA2 = 1.0, 0.0
ALPHA3, BETA3 = 0.0, 1.0
ALPHA4, BETA4 = 1.0, 0.0

N, K, ALPHA, MU, T = (128, 128), 1024, array([1.0, 1.0]), 1.0, 0.125

In [ ]:
u = splitting_method(ALPHA, lambda x, y, t: f_x_y_t(*ALPHA, MU, x, y, t),
        L, N, T, K,
        ALPHA1, BETA1, ALPHA2, BETA2,
        ALPHA3, BETA3, ALPHA4, BETA4,
        lambda x, t: phi_1_x_t(*ALPHA, MU, x, t), lambda x, t: phi_2_x_t(*ALPHA, MU, x, t),
        lambda y, t: phi_3_y_t(*ALPHA, MU, y, t), lambda y, t: phi_4_y_t(*ALPHA, MU, y, t),
        lambda x, y: psi_x_y(*ALPHA, MU, x, y),
        Method.ALTERNATING_DIRECTION, lambda uk: print(uk))